In [1]:
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
from math import factorial

In [2]:
def savitzky_golay(fs, aud, cutoff_freuency, passes=1):
    z = signal.savgol_filter(aud, 100, 3)
    b, a = signal.butter(3, cutoff_freuency / fs)
    zi = signal.lfilter_zi(b, a)
    z, _ = signal.lfilter(b, a, z, zi = zi * z[0])
    return z

In [3]:
def low_pass_filter(aud, cutoff_frequency, fs):
    nyquist_frequency = fs / 2
    normalized_cutoff = cutoff_frequency / nyquist_frequency

    # Создание фильтра низких частот
    filter_order = 4
    normalized_filter = signal.butter(filter_order, normalized_cutoff, btype='low', analog=False)
    filtered_signal = signal.filtfilt(normalized_filter[0], normalized_filter[1], aud)

    return filtered_signal


In [4]:
def find_peaks(spec, freq, t, delta_t=0.1, delta_freq=50):
    peaks = set()

    t_increment = int(delta_t * len(t) / (t[-1] - t[0]))
    freq_increment = int(delta_freq * len(freq) / (freq[-1] - freq[0]))

    for i, freq_val in enumerate(freq):
        for j, time_val in enumerate(t):
            t_min = max(0, j - t_increment)
            t_max = min(len(t), j + t_increment + 1)
            freq_min = max(0, i - freq_increment)
            freq_max = min(len(freq), i + freq_increment + 1)

            is_peak = True

            for a in range(freq_min, freq_max):
                for b in range(t_min, t_max):
                    if (a != i or b != j) and spec[i, j] < spec[a, b]:
                        is_peak = False
                        break
                if not is_peak:
                    break

            if is_peak:
                peaks.add(time_val)

    return peaks

In [5]:
def to_format_wav(y):
    return np.int16(y / np.max(np.abs(y)) * 32000)


def plot_spectrogram(fs, aud, name):
    powerSpectrum, frequenciesFound, time, imageAxis = plt.specgram(aud, Fs=fs)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [s]')
    plt.title(f'spectrogram_{name}')
    plt.savefig(f'spectrogram_{name}.png')
    plt.clf()
    return powerSpectrum, frequenciesFound, time

In [6]:
def main():
    fs, aud = wav.read('git1.wav')
    aud = aud[:,0]
    noise_power = np.mean(aud ** 2)

    plot_spectrogram(fs, aud, 'sound')

    denoised = savitzky_golay(fs, aud, 1000)
    spec, freq, time = plot_spectrogram(fs, denoised, 'denoised_savitzky_golay')
    wav.write('denoised_savitzky_golay.wav', fs, to_format_wav(denoised))


    cutoff_frequency = 1000
    denoised_lowpass = low_pass_filter(aud, cutoff_frequency, fs)
    spec, freq, time = plot_spectrogram(fs, denoised_lowpass, 'denoised_lowpass')
    wav.write('denoised_lowpass.wav', fs, to_format_wav(denoised_lowpass))


    peaks = find_peaks(spec, freq, time)
    with open ('results.txt', 'w') as f:
        f.write("Number:" + str(len(peaks)) + '\n')
        for peak in peaks:
            f.write(str(peak) + '\n')



if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/matplotlib/axes/_axes.py:7773: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


<Figure size 640x480 with 0 Axes>